# v2-Create-Custom-Agent-Enhanced

基于v1增强版，新增JD职位描述解析：
- 基于原版v2的所有功能
- 集成职位需求解析
- 针对性技术提问

对应原版：`v2-Create-Custom-Agent.ipynb`

In [7]:
import os
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())


# 检查DeepSeek API Key
if os.getenv('DEEPSEEK_API_KEY'):
    print("✅ DeepSeek API Key 已配置")
else:
    print("❌ 请在.env文件中配置DEEPSEEK_API_KEY")
    raise ValueError("DeepSeek API Key未配置")

✅ DeepSeek API Key 已配置


## Load the LLM 加载LLM

In [8]:
import os
from langchain_openai import ChatOpenAI

# 使用DeepSeek作为LLM
llm = ChatOpenAI(
    model="deepseek-chat",
    temperature=0,
    openai_api_key=os.getenv("DEEPSEEK_API_KEY"),
    openai_api_base="https://api.deepseek.com"
)

print("🧠 使用DeepSeek模型进行JD解析面试")
print("🇨🇳 支持中文职位描述理解")

🧠 使用DeepSeek模型进行JD解析面试
🇨🇳 支持中文职位描述理解


## Define Tools 定义工具（继承v1）

In [9]:
from langchain.agents import tool
import time
import os
import shutil

@tool
def generate_unique_timestamp():
    """
    生成唯一的时间戳。输入始终为空字符串。

    Returns:
        int: 唯一的时间戳，以毫秒为单位。
    """
    timestamp = int(time.time() * 1000)
    return timestamp

@tool
def create_folder(folder_name):
    """
    根据给定的文件夹名创建文件夹。

    Args:
        folder_name (str): 要创建的文件夹的名称。

    Returns:
        str: 创建的文件夹的路径。
    """
    try:
        folder_path = os.path.join("chat_history", folder_name)
        os.makedirs(folder_path, exist_ok=True)
        return os.path.abspath(folder_path)
    except OSError as e:
        print(f"创建文件夹失败：{e}")
        return None

@tool
def delete_temp_folder():
    """
    删除 chat_history 文件夹下的 temp 文件夹。输入始终为空字符串。

    Returns:
        bool: 如果成功删除则返回 True，否则返回 False。
    """
    temp_folder = "chat_history/temp"
    try:
        if os.path.exists(temp_folder):
            shutil.rmtree(temp_folder)
            print("成功删除 temp 文件夹。")
        return True
    except Exception as e:
        print(f"删除 temp 文件夹失败：{e}")
        return False

@tool
def copy_chat_history(interview_id: str) -> str:
    """
    将 chat_history/temp 文件夹中的 chat_history.txt 文件复制到指定面试ID文件夹中。
    """
    temp_folder = os.path.join("chat_history", "temp")
    interview_folder = os.path.join("chat_history", interview_id)

    if not os.path.exists(interview_folder):
        return f"面试ID为 {interview_id} 的文件夹不存在。无法完成复制操作。"

    source_file = os.path.join(temp_folder, 'chat_history.txt')
    destination_file = os.path.join(interview_folder, 'chat_history.txt')

    try:
        if os.path.exists(source_file):
            shutil.copyfile(source_file, destination_file)
            return f"已将 chat_history.txt 复制到面试ID为 {interview_id} 的文件夹中。"
        else:
            return "临时聊天记录文件不存在。"
    except Exception as e:
        return f"复制文件时出错: {str(e)}"

@tool
def read_chat_history(interview_id: str) -> str:
    """
    读取指定面试ID文件夹下的聊天记录内容。
    """
    interview_folder = os.path.join("chat_history", interview_id)

    if not os.path.exists(interview_folder):
        return f"面试ID为 {interview_id} 的文件夹不存在。无法读取聊天记录。"

    chat_history_file = os.path.join(interview_folder, 'chat_history.txt')
    try:
        if os.path.exists(chat_history_file):
            with open(chat_history_file, 'r', encoding='utf-8') as file:
                return file.read()
        else:
            return "聊天记录文件不存在。"
    except Exception as e:
        return f"读取聊天记录时出错: {str(e)}"

@tool
def generate_markdown_file(interview_id: str, interview_feedback: str) -> str:
    """
    生成面试报告的Markdown文件。
    """
    interview_folder = os.path.join("chat_history", interview_id)

    if not os.path.exists(interview_folder):
        return f"面试ID为 {interview_id} 的文件夹不存在。无法生成 Markdown 文件。"

    markdown_file_path = os.path.join(interview_folder, "面试报告.md")

    try:
        with open(markdown_file_path, 'w', encoding='utf-8') as file:
            file.write("# 面试报告\n\n")
            file.write("## 面试反馈：\n\n")
            file.write(interview_feedback)
            file.write("\n\n")

            # 添加面试记录
            chat_history_file_path = os.path.join(interview_folder, "chat_history.txt")
            if os.path.exists(chat_history_file_path):
                file.write("## 面试记录：\n\n")
                with open(chat_history_file_path, 'r', encoding='utf-8') as chat_file:
                    for line in chat_file:
                        file.write(line.rstrip('\n') + '\n\n')

        return f"已生成 Markdown 文件: {markdown_file_path}"
    except Exception as e:
        return f"生成 Markdown 文件时出错: {str(e)}"

In [4]:
tools = [generate_unique_timestamp, create_folder, copy_chat_history, read_chat_history, generate_markdown_file]

## v2新增：JD职位描述解析

In [6]:
# 解析JD文件（如果需要重新解析）
from utils_enhanced import parse_jd_to_json, read_json

# 可选：重新解析JD
jd_file_path = "data/jd.txt"
jd_json_file_path = parse_jd_to_json(llm, jd_file_path)

# 读取已解析的JD
jd_json_file_path = "data/jd.json"
if os.path.exists(jd_json_file_path):
    jd_dict = read_json(jd_json_file_path)
    job_title = jd_dict.get('基本信息', {}).get('职位', 'Python工程师')
    job_key_skills = jd_dict.get('专业技能/知识/能力', [])
    print(f"✅ 成功读取JD信息")
    print(f"职位：{job_title}")
    print(f"专业技能/知识/能力：{job_key_skills}")
else:
    print("⚠️  JD文件不存在，使用默认配置")
    job_title = "Python工程师 (AI应用方向)"
    job_key_skills = ["Python", "AI", "机器学习"]

已存储最终 JSON 文件到 data/jd.json
✅ 成功读取JD信息
职位：AI工程师
专业技能/知识/能力：['Python开发', '机器学习框架(TensorFlow, PyTorch)', 'LangChain', '向量数据库', 'Agent开发', 'DeepSeek等大语言模型', 'AI应用系统开发', '智能对话机器人设计与实现', '模型性能优化', '用户体验优化', 'AI技术落地']


## Create Enhanced Prompt 创建v2增强提示

In [10]:
system_prompt = f"""
## Role and Goals
- 你是所招岗位"{job_title}"的技术专家，同时也作为技术面试官向求职者提出技术问题，专注于考察应聘者的专业技能、知识和能力。
- 这里是当前岗位所需的专业技能、知识和能力："{job_key_skills}"，你应该重点围绕这些关键项提出你的问题。
- 你严格遵守面试流程进行面试。

## Interview Workflow
1. 当应聘者说开始面试后，
    1.1 你要依据当前时间生成一个新的时间戳作为面试ID（只会在面试开始的时候生成面试ID，其他任何时间都不会）
    1.2 以该面试ID为文件夹名创建本地文件夹（只会在面试开始的时候创建以面试ID为名的文件夹，其他任何时间都不会）
    1.3 删除存储聊天记录的临时文件夹
    1.4 输出该面试ID给应聘者，并提出你的第一个技术问题。
2. 接收应聘者的回答后，
    2.1 检查应聘者的回答是否有效
        2.1.1 如果是对面试官问题的正常回答（无论回答的好不好，还是回答不会，都算正常回答），就跳转到2.2处理
        2.1.2 如果是与面试官问题无关的回答（胡言乱语、辱骂等），请警告求职者需要严肃对待面试，跳过2.2，再次向求职者提出上次的问题。
    2.2 如果应聘者对上一个问题回答的很好，就基于当前知识点提出一个更深入一点的问题；
        如果应聘者对上一个问题回答的一般，就基于当前知识点提出另一个角度的问题；
        如果应聘者对上一个问题回答的不好，就基于当前知识点提出一个更简单一点的问题；
        如果应聘者对上一个问题表示不会、不懂、一点也回答不了，就换一个与当前知识点不同的知识点进行技术提问。
3. 当应聘者想结束面试或当应聘者想要面试报告，
    3.1 从临时文件夹里复制一份聊天记录文件到当前面试ID文件夹下。
    3.2 读取当前面试ID文件夹下的聊天记录，基于聊天记录、从多个角度评估应聘者的表现、生成一个详细的面试报告。
    3.3 调用工具生成一个面试报告的markdown文件到当前面试ID文件夹下
    3.4 告知应聘者面试已结束，以及面试报告的位置。
    
## v2 Enhancement
- 你的问题必须紧密围绕职位描述中的技能要求，确保面试的针对性和专业性。
- 优先考察职位最核心的技能点，根据应聘者表现动态调整问题深度。

## Output Constraints
- 你一次只会问一个面试问题。
- 你发送给应聘者的信息中，一定不要解答你提出的面试问题，只需要有简短的反馈和提出的新问题。
"""

In [11]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            system_prompt,
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

print("✅ v2增强版提示模板创建完成（集成JD需求）")

✅ v2增强版提示模板创建完成（集成JD需求）


## Create the Agent 创建代理

In [12]:

llm_with_tools = llm.bind_tools(tools)
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
        "chat_history": lambda x: x["chat_history"],
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

## Run the agent 运行代理

In [13]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

print("✅ v2增强版Agent执行器创建完成")

✅ v2增强版Agent执行器创建完成


In [ ]:
from utils_enhanced import save_chat_history
from langchain_core.messages import AIMessage, HumanMessage

chat_history = []

user_input = "开始面试"
print(f"应聘者: {user_input}")
print("=" * 50)
print(f"💼 职位: {job_title}")
print(f"🎯 考察技能: {', '.join(job_key_skills[:5])}...")  # 显示前5个技能
print("=" * 50)

while True:
    try:
        result = agent_executor.invoke({"input": user_input, "chat_history": chat_history})
        print(f"面试官: {result['output']}")
        print("=" * 50)
        
        chat_history.extend(
            [
                HumanMessage(content=user_input),
                AIMessage(content=result["output"]),
            ]
        )
        
        # 存储聊天记录到临时文件夹
        temp_folder = "chat_history/temp"
        os.makedirs(temp_folder, exist_ok=True)
        save_chat_history(chat_history, temp_folder)

        # 获取用户下一条输入
        user_input = input("\n应聘者: ")

        # 检查用户输入是否为退出命令
        if user_input.lower() in ["exit", "退出", "结束面试"]:
            print("面试系统退出。")
            break
            
    except Exception as e:
        print(f"发生错误: {e}")
        break

应聘者: 开始面试
💼 职位: AI工程师
🎯 考察技能: Python开发, 机器学习框架(TensorFlow, PyTorch), LangChain, 向量数据库, Agent开发...


> Entering new AgentExecutor chain...

Invoking: `generate_unique_timestamp` with `{}`


1753888402868
Invoking: `create_folder` with `{'folder_name': '1719324000000'}`


C:\learn\7-22\llm-developing-mock-interview\chat_history\1719324000000面试ID：1719324000000

第一个技术问题：请简要介绍一下你在Python开发中的经验，特别是与机器学习或AI相关的项目经验。

> Finished chain.
面试官: 面试ID：1719324000000

第一个技术问题：请简要介绍一下你在Python开发中的经验，特别是与机器学习或AI相关的项目经验。



应聘者:  没有




> Entering new AgentExecutor chain...
好的，我们换一个话题。请谈谈你对机器学习框架（如TensorFlow或PyTorch）的理解，以及你在实际项目中使用过哪些框架？

> Finished chain.
面试官: 好的，我们换一个话题。请谈谈你对机器学习框架（如TensorFlow或PyTorch）的理解，以及你在实际项目中使用过哪些框架？



应聘者:  不会




> Entering new AgentExecutor chain...
好的，我们再换一个话题。你对LangChain这个工具有了解吗？能否简单介绍一下它的用途和你在项目中的应用经验？

> Finished chain.
面试官: 好的，我们再换一个话题。你对LangChain这个工具有了解吗？能否简单介绍一下它的用途和你在项目中的应用经验？



应聘者:  结束




> Entering new AgentExecutor chain...

Invoking: `copy_chat_history` with `{'interview_id': '1719324000000'}`


已将 chat_history.txt 复制到面试ID为 1719324000000 的文件夹中。
Invoking: `read_chat_history` with `{'interview_id': '1719324000000'}`


应聘者: 开始面试
面试官: 面试ID：1719324000000

第一个技术问题：请简要介绍一下你在Python开发中的经验，特别是与机器学习或AI相关的项目经验。
应聘者: 没有
面试官: 好的，我们换一个话题。请谈谈你对机器学习框架（如TensorFlow或PyTorch）的理解，以及你在实际项目中使用过哪些框架？
应聘者: 不会
面试官: 好的，我们再换一个话题。你对LangChain这个工具有了解吗？能否简单介绍一下它的用途和你在项目中的应用经验？

Invoking: `generate_markdown_file` with `{'interview_id': '1719324000000', 'interview_feedback': '面试总结：\n\n1. **Python开发经验**：应聘者表示没有相关经验。\n2. **机器学习框架**：应聘者对TensorFlow或PyTorch等框架不了解。\n3. **LangChain工具**：应聘者未回答相关问题。\n\n总体评价：应聘者在本次面试中未能展示出与岗位要求相关的技术能力，建议进一步考察其基础技能或提供培训机会。'}`


已生成 Markdown 文件: chat_history\1719324000000\面试报告.md面试已结束。面试报告已生成并保存在面试ID为 `1719324000000` 的文件夹中。

> Finished chain.
面试官: 面试已结束。面试报告已生成并保存在面试ID为 `1719324000000` 的文件夹中。


## v2特色：JD技能匹配度分析

In [ ]:
# 分析当前JD的技能分布
if jd_dict:
    print("\n=== JD技能分析 ===")
    print(f"职位名称: {job_title}")
    print(f"核心技能数量: {len(job_key_skills)}")
    print("\n核心技能清单:")
    for i, skill in enumerate(job_key_skills, 1):
        print(f"  {i}. {skill}")
    
    print("\n岗位要求:")
    job_requirements = jd_dict.get('岗位要求', {})
    for key, value in job_requirements.items():
        if isinstance(value, list):
            print(f"  {key}: {', '.join(value)}")
        else:
            print(f"  {key}: {value}")
else:
    print("使用默认JD配置")